In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from huggingface_hub import login

login("my_hf_token")

# Check GPU availability
print(f"CUDA Available: {torch.cuda.is_available()}")
print(f"GPU: {torch.cuda.get_device_name(0)}")

# Load tokenizer and model
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3.2-1B", use_auth_token=True)
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-3.2-1B",
    device_map="auto",
    use_auth_token=True
)

print("LLaMA 3 loaded successfully!")

/home/hice1/jgustin31/scratch/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CUDA Available: True
GPU: NVIDIA H100 80GB HBM3


/home/hice1/jgustin31/scratch/venv/lib/python3.10/site-packages/transformers/models/auto/tokenization_auto.py:1041: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/home/hice1/jgustin31/scratch/venv/lib/python3.10/site-packages/transformers/models/auto/auto_factory.py:492: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


LLaMA 3 loaded successfully!


In [2]:
print("Saving Pretrained model...")
SAVE_PATH = "./llama3_finetuned_5_epoch_all_new/pretrained_model"
model.save_pretrained(SAVE_PATH)
tokenizer.save_pretrained(SAVE_PATH)

print(f"Pretrained model and tokenizer saved successfully to: {SAVE_PATH}")

Saving Pretrained model...
Pretrained model and tokenizer saved successfully to: ./llama3_finetuned_5_epoch_all_new/pretrained_model


In [3]:
def run_demo_examples():
    """
    Runs two predefined prompts through the globally loaded model/tokenizer
    and prints the outputs with separators.
    """

    prompts = [
        "Explain the concept of black holes.",

        """Question: Which driver secured a double victory at the US Grand Prix held in Austin?

Options:
A: Sergio Pérez
B: Lewis Hamilton
C: Max Verstappen
D: Charles Leclerc

Correct option:"""
    ]

    for i, prompt in enumerate(prompts, start=1):
        print("\n" + "="*50)
        print(f"Example {i}:")
        print("="*50)
        print("Prompt:")
        print(prompt)
        print("-"*50)

        # Tokenize
        inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

        # Generate
        with torch.no_grad():
            output_ids = model.generate(**inputs,max_new_tokens=300)

        # Decode
        output_text = tokenizer.decode(output_ids[0], skip_special_tokens=True)

        # Print response
        print("Response:")
        print(output_text)
        print("="*50 + "\n")


In [4]:
run_demo_examples()

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Example 1:
Prompt:
Explain the concept of black holes.
--------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Response:
Explain the concept of black holes. (10 marks)
Explain the concept of black holes. (10 marks)
Black holes are regions of space-time where gravitational fields are so strong that nothing can escape. The gravitational field of a black hole is so strong that even light cannot escape. This is because the gravitational field of a black hole is so strong that even light cannot escape. This is because the gravitational field of a black hole is so strong that even light cannot escape. This is because the gravitational field of a black hole is so strong that even light cannot escape.
Explain the concept of black holes. (10 marks)
Black holes are regions of space-time where gravitational fields are so strong that nothing can escape. The gravitational field of a black hole is so strong that even light cannot escape. This is because the gravitational field of a black hole is so strong that even light cannot escape. This is because the gravitational field of a black hole is so strong that

In [5]:
DATASET_DIR = "./dataset"  
import re
import os
import json
import re
from glob import glob

def build_input_text(question: str, options: dict) -> str:
    """
    Build the input string in the exact format you showed.
    """
    return (
        f"Question: {question}\n\n"
        f"Options:\n"
        f"A: {options['A']}\n"
        f"B: {options['B']}\n"
        f"C: {options['C']}\n"
        f"D: {options['D']} \n"
        f"Correct option:"
    )


def extract_model_option(output_text: str) -> str | None:
    """
    Extract the option letter (A/B/C/D) from the model output.

    Strategy:
    1. Look for 'Correct option: X' (case-insensitive).
    2. If not found, try to find a standalone A/B/C/D on the last lines.
    """

    # 1. Regex for "Correct option: X"
    matches = re.findall(r"Correct option\s*:\s*([A-D])", output_text, flags=re.IGNORECASE)
    if matches:
        return matches[-1].upper()  # take the last occurrence

    # 2. Fallback: search last few lines for an isolated A-D
    lines = [l.strip() for l in output_text.splitlines() if l.strip()]
    for line in reversed(lines[-5:]):  # look at the last few non-empty lines
        m = re.match(r"^[A-D]$", line.strip(), flags=re.IGNORECASE)
        if m:
            return m.group(0).upper()

    return None  # couldn't parse


def evaluate_dataset(dataset_dir: str):
    json_files = glob(os.path.join(dataset_dir, "*.json"))

    total = 0
    correct = 0
    skipped = 0  

    number = 1
    for json_path in json_files:
        with open(json_path, "r", encoding="utf-8") as f:
            data = json.load(f)

        qa_pairs = data.get("qa_pairs", [])
        for qa in qa_pairs:
            # Prefer rephrased_question if available, else fall back to question
            question = qa.get("question")
            options = qa["options"]
            ground_truth = qa["ground_truth_correct_option"].strip().upper()

            input_text = build_input_text(question, options)

            # Prepare model input
            inputs = tokenizer(input_text, return_tensors="pt").to("cuda")
            with torch.no_grad():
                outputs = model.generate(
                    **inputs,
                    max_length=1000,
                    do_sample=False  # deterministic; change if you want sampling
                )
            decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

            model_option = extract_model_option(decoded)

            total += 1
            if model_option is None:
                skipped += 1
                print(f"[WARNING] Could not parse model output for question:\n{question}\n")
                print(f"Raw output:\n{decoded}\n{'-'*80}")
                continue

            if model_option == ground_truth:
                correct += 1
            
            print(number, ": ", correct, "/", total, " (skipped:", skipped, ")")
            number += 1

    accuracy = correct / total if total > 0 else 0.0
    print(f"Total questions: {total}")
    print(f"Correct: {correct}")
    print(f"Skipped (unparsable outputs): {skipped}")
    print(f"Accuracy: {accuracy * 100:.2f}%")

    return accuracy

evaluate_dataset(DATASET_DIR)

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


1 :  0 / 1  (skipped: 0 )
2 :  0 / 2  (skipped: 0 )
3 :  0 / 3  (skipped: 0 )
4 :  0 / 4  (skipped: 0 )
5 :  1 / 5  (skipped: 0 )
6 :  1 / 6  (skipped: 0 )


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


7 :  2 / 7  (skipped: 0 )
8 :  2 / 8  (skipped: 0 )
9 :  3 / 9  (skipped: 0 )


...

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


1566 :  553 / 1566  (skipped: 0 )
1567 :  553 / 1567  (skipped: 0 )
1568 :  553 / 1568  (skipped: 0 )
1569 :  553 / 1569  (skipped: 0 )
1570 :  553 / 1570  (skipped: 0 )
1571 :  553 / 1571  (skipped: 0 )
1572 :  553 / 1572  (skipped: 0 )
1573 :  553 / 1573  (skipped: 0 )


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


1574 :  554 / 1574  (skipped: 0 )
1575 :  554 / 1575  (skipped: 0 )
1576 :  555 / 1576  (skipped: 0 )
1577 :  555 / 1577  (skipped: 0 )
1578 :  556 / 1578  (skipped: 0 )
1579 :  556 / 1579  (skipped: 0 )
1580 :  556 / 1580  (skipped: 0 )
1581 :  556 / 1581  (skipped: 0 )
Total questions: 1581
Correct: 556
Skipped (unparsable outputs): 0
Accuracy: 35.17%


0.35167615433270083

In [6]:
import os
import json
from datasets import Dataset, DatasetDict

import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from trl import SFTTrainer, SFTConfig


DATASET_DIR = "dataset"     # folder containing 512 json files
PROMPT_TEMPLATE = """Below is a question about Formula 1. Your task is to provide the correct answer clearly and concisely.

Question: {question}

Answer:
"""

def load_qa_data(data_dir):
    records = []

    for i in range(1, 513):
        file_path = os.path.join(data_dir, f"qa_{i}.json")
        if not os.path.exists(file_path):
            print(f"Skipping missing file: {file_path}")
            continue

        with open(file_path, "r", encoding="utf-8") as f:
            data = json.load(f)

        for qa in data["qa_pairs"]:
            question = qa["question"]
            answer = qa["correct_answer"]

            prompt = PROMPT_TEMPLATE.format(question=question)

            records.append({
                "question": prompt,
                "answer": answer,
            })

    return records


def preprocess(example):
    inputs = tokenizer(example["question"], max_length=256, truncation=False, padding="max_length")
    targets = tokenizer(example["answer"], max_length=256, truncation=False, padding="max_length")
    inputs["labels"] = targets["input_ids"]
    return inputs

records = load_qa_data(DATASET_DIR)
dataset = Dataset.from_list(records)
tokenizer.pad_token = tokenizer.eos_token
tokenized_dataset = dataset.map(preprocess)
tokenized_dataset.set_format("torch", columns=["input_ids", "attention_mask", "labels"])

print(tokenized_dataset)



Map: 100%|██████████| 1536/1536 [00:00<00:00, 2820.37 examples/s]

Dataset({
    features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 1536
})


In [7]:
print(tokenized_dataset)

Dataset({
    features: ['question', 'answer', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 1536
})


In [8]:
print(tokenized_dataset[0])

{'input_ids': tensor([128000,  39314,    374,    264,   3488,    922,  31922,    220,     16,
            13,   4718,   3465,    374,    311,   3493,    279,   4495,   4320,
          9539,    323,   3613,    285,    989,    382,  14924,     25,  16299,
          5696,  17427,    264,   2033,   3243,    520,    279,   2326,  10517,
         44394,    304,  19816,   1980,  16533,    512, 128001, 128001, 128001,
        128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001,
        128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001,
        128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001,
        128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001,
        128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001,
        128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001,
        128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001, 128001,
        128001, 128001, 12

In [9]:
from transformers import Seq2SeqTrainingArguments, TrainingArguments


training_args = Seq2SeqTrainingArguments(
    output_dir="./llama3_finetuned_5_epoch_all_new",
    per_device_train_batch_size=16,
    num_train_epochs=100,
    logging_steps=1,
    save_strategy="steps",
    save_steps=960,
    push_to_hub=False,
    learning_rate=1e-2,
    eval_strategy="epoch",
)

In [10]:
from transformers import Trainer

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset.select(range(20)),
)

trainer.train()

The model is already on multiple devices. Skipping the move to device specified in `args`.


Epoch,Training Loss,Validation Loss
1,1.684200,1.592747
2,0.314800,0.144477
3,0.169400,0.114440
4,0.160000,0.095399
5,0.194200,0.108283
6,0.174600,0.103527
7,0.192300,0.102772
8,0.118500,0.099428
9,0.114100,0.096367
10,0.150400,0.094242


TrainOutput(global_step=9600, training_loss=0.1994710284541361, metrics={'train_runtime': 6816.9686, 'train_samples_per_second': 22.532, 'train_steps_per_second': 1.408, 'total_flos': 2.295939729457152e+17, 'train_loss': 0.1994710284541361, 'epoch': 100.0})

In [11]:
run_demo_examples()



Example 1:
Prompt:
Explain the concept of black holes.
--------------------------------------------------


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Response:
Explain the concept of black holes.


Example 2:
Prompt:
Question: Which driver secured a double victory at the US Grand Prix held in Austin?

Options:
A: Sergio Pérez
B: Lewis Hamilton
C: Max Verstappen
D: Charles Leclerc

Correct option:
--------------------------------------------------
Response:
Question: Which driver secured a double victory at the US Grand Prix held in Austin?

Options:
A: Sergio Pérez
B: Lewis Hamilton
C: Max Verstappen
D: Charles Leclerc

Correct option:



In [12]:
# ============================================================
# SAVE MODEL
# ============================================================

print("Saving model...")
OUTPUT_DIR = "./llama3_finetuned_5_epoch_all_new/final_finetuned_model_100"
trainer.save_model(OUTPUT_DIR)
tokenizer.save_pretrained(OUTPUT_DIR)

print("Fine-tuning complete! Model saved to:", OUTPUT_DIR)

Saving model...
Fine-tuning complete! Model saved to: ./llama3_finetuned_5_epoch_all_new/final_finetuned_model_100
